<a href="https://colab.research.google.com/github/RZdataprojects/Data-Mining---Rain-and-Bloom/blob/main/Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining

Roei Zaady


318747946


Ben-Gurion University of the Negev

Packages used: ee (google earth engine), pandas, numpy, pyproj (maps), folium (maps), geemap (maps), datetime, duckdb (SQL), sklearn, xgboost, shap, plotly.

## Imports and Data

In [ ]:
import ee
import pandas as pd
from pyproj import Proj, transform
import folium
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
!pip install -U gdown

!gdown 1N0qlqyCbHu2aIvmW7NLmEVueb87tNhKt
!gdown 1N3ia-hJVSIUXLQYJ_WJzni33F_REi436
!gdown 1gYdg1N6V74A6uc_R99pIfFPGfA3T4gnG
!gdown 1MxO35R9qEva5fws0T7o1HxwIwMGcfeQ5
!gdown 1MwzXLhJm31brAS0Yq6E08i0y-Pz7BFBE
!gdown 1QbizAjNlhMThAw9TXJABJrBQUDt0a4L2

Downloading...
From: https://drive.google.com/uc?id=1N0qlqyCbHu2aIvmW7NLmEVueb87tNhKt
To: /content/df with lat_long.csv
100% 274k/274k [00:00<00:00, 106MB/s]
Downloading...
From: https://drive.google.com/uc?id=1N3ia-hJVSIUXLQYJ_WJzni33F_REi436
To: /content/ndvi_results.csv
100% 36.5M/36.5M [00:00<00:00, 66.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gYdg1N6V74A6uc_R99pIfFPGfA3T4gnG
To: /content/IMS_2002-2023.zip
100% 10.9M/10.9M [00:00<00:00, 76.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MxO35R9qEva5fws0T7o1HxwIwMGcfeQ5
To: /content/pre-processed point.csv
100% 194k/194k [00:00<00:00, 77.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1MwzXLhJm31brAS0Yq6E08i0y-Pz7BFBE
From (redirected): https://drive.google.com/uc?id=1MwzXLhJm31brAS0Yq6E08i0y-Pz7BFBE&confirm=t&uuid=4abf4681-02a5-46ed-b015-4d18ff2c6f32
To: /content/final_ds.csv
100% 151M/151M [00:00<00:00, 164MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbizAjNlhMThAw9

OBJECTID - Location ID


INDEX - Row ID


Slope_mt_neg2 - Slope (per what unit?)


mt_negev - Elevation


Litho - Stone Composition


category - "Natural and Wooded" - identical to all rows


FNAME1 - Name in Hebrew


FTYPE_DESC - 'מעביר מים', 'נחל צר', 'נחל רחב', 'ערוצון/בתרון', 'תעלה'


STRM_ORD_1 - ?


FSTATUS_DE - Type of Stream ('זרימה בעיתות גשם - אכזב', 'מים', 'ניקוז', 'ערוץ מדומה')


POINT_X - latitude based on the Israeli TM Grid EPSG:2039


POINT_Y - longitude based on the Israeli TM Grid EPSG:2039

## Conversion from the Israeli TM Grid EPSG:2039 projection to Google's Earth prefered projection EPSG:4326

In [ ]:
def convert_coordinates(x, y):
    # Define the EPSG:2039 and EPSG:4326 projections using pyproj Transformer
    proj2039 = Proj(init='epsg:2039')
    proj4326 = Proj(init='epsg:4326')

    # Convert EPSG:2039 to EPSG:4326
    lon, lat = transform(proj2039, proj4326, x, y)
    return lat, lon

# Apply the convert_coordinates function to create new columns
#df[['Latitude', 'Longitude']] = df.apply(lambda row: pd.Series(convert_coordinates(row['POINT_X'], row['POINT_Y'])), axis=1)
df_ = pd.read_csv('/content/df with lat_long.csv', index_col='INDEX')

## Checking for Missing Values

In [ ]:
df_ = pd.read_csv('/content/df with lat_long.csv', index_col='INDEX')
display(df_.head(1))

,OBJECTID,Slope_mt_neg2,mt_negev,Litho,category,FNAME1,FTYPE_DESC,STRM_ORD_1,FSTATUS_DE,POINT_X,POINT_Y,Latitude,Longitude
INDEX,,,,,,,,,,,,,
1,1,7.235941,617.238037,Limestone/Dolomite/Marl,Natural and Wooded,,נחל צר,0.0,זרימה בעיתות גשם - אכזב,172983.9243,519275.619,30.763107,34.719044


In [ ]:
df_.nunique()

OBJECTID         1431
Slope_mt_neg2    1431
mt_negev         1431
Litho               6
category            1
FNAME1             36
FTYPE_DESC          5
STRM_ORD_1          7
FSTATUS_DE          4
POINT_X          1431
POINT_Y          1431
Latitude         1431
Longitude        1431
dtype: int64

In [ ]:
df_.FTYPE_DESC.value_counts()

FTYPE_DESC
נחל צר          1296
נחל רחב          101
תעלה              10
ערוצון/בתרון       9
מעביר מים          3
Name: count, dtype: int64

In [ ]:
df_.STRM_ORD_1.value_counts()

STRM_ORD_1
1.0    615
0.0    487
2.0    187
3.0     67
4.0     46
6.0      9
5.0      8
Name: count, dtype: int64

In [ ]:
df_.Litho.value_counts()

Litho
Limestone/Dolomite/Marl    1002
Chalk                       204
Alluvium                    126
Chert                        80
Nubian sandstone             10
Conglomerate                  9
Name: count, dtype: int64

In [ ]:
df_.FSTATUS_DE.value_counts()

FSTATUS_DE
זרימה בעיתות גשם - אכזב    1404
מים                           9
ניקוז                         4
ערוץ מדומה                    2
Name: count, dtype: int64

In [ ]:
df_.columns = ['OBJECTID',	'Slope', 	'Elevation', 	'Litho', 	'category' ,	'FNAME1' ,	'FTYPE' 	,'STRM_ORD_1' ,	'FSTATUS_DE' ,	'POINT_X' ,	'POINT_Y' ,	'Latitude' ,	'Longitude']
df_.head(1)

,OBJECTID,Slope,Elevation,Litho,category,FNAME1,FTYPE,STRM_ORD_1,FSTATUS_DE,POINT_X,POINT_Y,Latitude,Longitude
INDEX,,,,,,,,,,,,,
1,1,7.235941,617.238037,Limestone/Dolomite/Marl,Natural and Wooded,,נחל צר,0.0,זרימה בעיתות גשם - אכזב,172983.9243,519275.619,30.763107,34.719044


## Remove Useless Features

In [ ]:
df_.drop(columns=['category', 'FNAME1', 'FSTATUS_DE', 'POINT_X', 'POINT_Y'], inplace=True)

## One-Hot Encoding

In [ ]:
df_.drop(columns=['category', 'FNAME1', 'FSTATUS_DE', 'POINT_X', 'POINT_Y'], inplace=True)
# One-hot encode the 'Litho' and 'FTYPE' columns
df_ = pd.get_dummies(df_, columns=['Litho', 'FTYPE'])
df_.head()

,OBJECTID,Slope,Elevation,STRM_ORD_1,Latitude,Longitude,Litho_Alluvium,Litho_Chalk,Litho_Chert,Litho_Conglomerate,Litho_Limestone/Dolomite/Marl,Litho_Nubian sandstone,FTYPE_מעביר מים,FTYPE_נחל צר,FTYPE_נחל רחב,FTYPE_ערוצון/בתרון,FTYPE_תעלה
INDEX,,,,,,,,,,,,,,,,,
1,1,7.235941,617.238037,0.0,30.763107,34.719044,False,False,False,False,True,False,False,True,False,False,False
3,2,8.317689,597.649170,5.0,30.745758,34.708510,False,False,False,False,True,False,False,False,True,False,False
4,3,4.505186,635.804443,0.0,30.779991,34.691815,False,False,False,False,True,False,False,True,False,False,False
5,4,6.490717,625.075806,1.0,30.749266,34.694740,False,False,False,False,True,False,False,False,True,False,False
6,5,18.905861,591.240784,4.0,30.765798,34.723000,False,False,False,False,True,False,False,False,True,False,False


## Draw Map with Points

In [ ]:
def add_points(map_obj, dataframe):
    for idx, row in dataframe.iterrows():
        lat, lon = row['Latitude'], row['Longitude']
        popup_content = f"""
        <strong>Name:</strong> {row['FNAME1']}<br>
        <strong>Longitude:</strong> {lon}<br>
        <strong>Latitude:</strong> {lat}<br>
        """
        folium.Marker([lat, lon], popup=popup_content).add_to(map_obj)

# Create a folium map centered at Sde Boker coordinates
sde_boker_lat, sde_boker_lon = 30.869830804375287, 34.79422640660065
map_center = [30.905, 34.79422640660065]
m = folium.Map(location=map_center, zoom_start=12)

# Add marker for Sde Boker Weather Station
popup_sb = "<strong>Sde Boker Meteorological Station</strong><br>Coordinates: 30.8702, 34.7950"
folium.Marker([sde_boker_lat, sde_boker_lon], popup=popup_sb, icon=folium.Icon(color='red')).add_to(m)

# Add points from the DataFrame
add_points(m, df_)

# Display the map
m

## Getting Satelite Imagery (NDVI)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Assuming df is your DataFrame with columns ['Latitude'], ['Longitude']
# Create a date range from '2002-05-01' to '2024-01-01'
start_date = datetime(2002, 10, 30)
end_date = datetime(2024, 1, 1)
date_range = pd.date_range(start=start_date, end=end_date, freq='15D')

# Repeat the date range for each pair of Latitude and Longitude
dates_per_pair = len(date_range)
total_pairs = len(df)
dates = date_range.tolist() * total_pairs

# Create a new DataFrame with dates, Latitude, and Longitude
new_data = {
    'OBJECTID': df['OBJECTID'].repeat(dates_per_pair).reset_index(drop=True),
    'Date': dates[:dates_per_pair * total_pairs],
    'Latitude': df['Latitude'].repeat(dates_per_pair).reset_index(drop=True),
    'Longitude': df['Longitude'].repeat(dates_per_pair).reset_index(drop=True)
}
new_df = pd.DataFrame(new_data)

# Display the new DataFrame
display(new_df)

In [ ]:
import numpy as np

# Define a function to convert DataFrame points to FeatureCollection
def convert_to_feature(point):
    return ee.Feature(ee.Geometry.Point(point['Longitude'], point['Latitude']))

# Define a function to calculate NDVI metrics for multiple points
def calculate_ndvi_metrics(point, start_date, end_date):
    # Convert points to a FeatureCollection
    features = ee.FeatureCollection(convert_to_feature(point))

    # Filter the image collection by date and bounds
    filtered_collection = ndvi_collection.filterBounds(features).filterDate(start_date, end_date)

    # Calculate NDVI
    def calculate_ndvi(image):
        ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
        return image.addBands(ndvi)

    # Map NDVI calculation over the filtered collection
    ndvi_with_band = filtered_collection.map(calculate_ndvi)

    # Reduce the collection to get mean NDVI for each image
    ndvi_mean = ndvi_with_band.reduce(ee.Reducer.mean())

    # Get the NDVI values as a time series
    ndvi_values = ndvi_mean.select('NDVI_mean').reduceRegions(
        collection=features,
        reducer=ee.Reducer.mean(),
        scale=250,  # Adjust scale as needed
      )

    ndvi = ndvi_values.getInfo()['features'][0]['properties']['mean']
    #print(np.array([end_date, point.OBJECTID, point['Longitude'], point['Latitude'], ndvi]))
    #df_results = pd.DataFrame(ndvi_values.getInfo()['features'])
    return np.array([end_date, point.OBJECTID, point['Longitude'], point['Latitude'], ndvi])

# Load MODIS NDVI Image Collection
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13Q1')

# Process each batch and concatenate results
df_results_list = []
for _,point in new_df.iterrows():
    print(_)
    if _ % 150000 == 0:
        df = pd.DataFrame(df_results_list, columns=['Date', 'OBJECTID', 'Longitude', 'Latitude', 'NDVI'])
        df.to_csv(str(_) + '_.csv', index=False)
    start_date = str(point.Date)[:10]
    end_date = str((point.Date + pd.Timedelta(days=14)).date())
    try:
        df_results_batch = calculate_ndvi_metrics(point, start_date, end_date)
        df_results_list.append(df_results_batch)
    except:
        pass

df = pd.DataFrame(df_results_list, columns=['Date', 'OBJECTID', 'Longitude', 'Latitude', 'NDVI'])
# Export DataFrame to CSV or desired format
df.to_csv('ndvi_results.csv', index=False)

## NDVI Vegetation Map Animation

In [ ]:
import geemap
Map = geemap.Map()

# Define the region of interest (ROI)
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.Rectangle([34, 29.2, 36, 33.5])
    Map.addLayer(roi)
    Map.centerObject(roi)

# Create the MODIS NDVI timelapse
timelapse = geemap.modis_ndvi_timelapse(
    roi,
    out_gif="ndvi.gif",
    data="Terra",
    band="NDVI",
    start_date="2002-01-01",
    end_date="2023-12-31",
    frames_per_second=3,
    title="MODIS NDVI Timelapse (%Y)",
    overlay_data="countries",
)

# Display the timelapse
geemap.show_image(timelapse)

## Combaining Data Sets with duckdb (SQL)

In [ ]:
import pandas as pd

df2 = pd.read_csv('/content/ndvi_results.csv')
df2['Start Date'] = pd.to_datetime(df2['Date']) - pd.DateOffset(days=14)
df2['Start Date'] = df2['Start Date'].dt.strftime('%Y-%m-%d')
df2['Date'] = pd.to_datetime(df2['Date'])
df2['Start Date'] = pd.to_datetime(df2['Start Date'])
display(df2)


,Date,OBJECTID,Longitude,Latitude,NDVI,Start Date
0,2002-11-13,1,34.719044,30.763107,995.0,2002-10-30
1,2002-11-28,1,34.719044,30.763107,1024.0,2002-11-14
2,2002-12-13,1,34.719044,30.763107,1188.0,2002-11-29
3,2002-12-28,1,34.719044,30.763107,1249.0,2002-12-14
4,2003-01-12,1,34.719044,30.763107,1195.0,2002-12-29
...,...,...,...,...,...,...
645640,2023-11-08,1431,34.901230,30.941668,1052.0,2023-10-25
645641,2023-11-23,1431,34.901230,30.941668,1062.0,2023-11-09
645642,2023-12-08,1431,34.901230,30.941668,1157.0,2023-11-24
645643,2023-12-23,1431,34.901230,30.941668,1187.0,2023-12-09


In [ ]:
df2 = pd.merge(left=df2, right=df_, left_on='OBJECTID', right_on='OBJECTID')
df2.drop(columns=['Latitude_y', 'Longitude_y'], inplace=True)
df2.columns= ['Date', 'OBJECTID', 'Longitude', 'Latitude', 'NDVI', 'Start Date',
       'Slope', 'Elevation', 'STRM_ORD_1', 'Litho_Alluvium', 'Litho_Chalk',
       'Litho_Chert', 'Litho_Conglomerate', 'Litho_Limestone/Dolomite/Marl',
       'Litho_Nubian sandstone', 'FTYPE_מעביר מים', 'FTYPE_נחל צר',
       'FTYPE_נחל רחב', 'FTYPE_ערוצון/בתרון', 'FTYPE_תעלה']
df2.head(1)

,Date,OBJECTID,Longitude,Latitude,NDVI,Start Date,Slope,Elevation,STRM_ORD_1,Litho_Alluvium,Litho_Chalk,Litho_Chert,Litho_Conglomerate,Litho_Limestone/Dolomite/Marl,Litho_Nubian sandstone,FTYPE_מעביר מים,FTYPE_נחל צר,FTYPE_נחל רחב,FTYPE_ערוצון/בתרון,FTYPE_תעלה
0,2002-11-13,1,34.719044,30.763107,995.0,2002-10-30,7.235941,617.238037,0.0,False,False,False,False,True,False,False,True,False,False,False


In [ ]:
df2.columns

Index(['Date', 'OBJECTID', 'Longitude', 'Latitude', 'NDVI', 'Start Date',
       'Slope', 'Elevation', 'STRM_ORD_1', 'Litho_Alluvium', 'Litho_Chalk',
       'Litho_Chert', 'Litho_Conglomerate', 'Litho_Limestone/Dolomite/Marl',
       'Litho_Nubian sandstone', 'FTYPE_מעביר מים', 'FTYPE_נחל צר',
       'FTYPE_נחל רחב', 'FTYPE_ערוצון/בתרון', 'FTYPE_תעלה'],
      dtype='object')

In [ ]:
df1 = pd.read_csv('/content/grouped_df.csv')
df1['Date'] = pd.to_datetime(df1['Date'])
display(df1)

In [ ]:
import duckdb

df = duckdb.query("""
                  SELECT
                  df2['Start Date'],
                  df2['Date'] as 'End Date',
                  df2.Longitude,
                  df2.Latitude,
                  Slope,
                  Elevation,
                  STRM_ORD_1,
                  Litho_Alluvium,
                  Litho_Chalk,
                  Litho_Chert,
                  Litho_Conglomerate,
                  ['Litho_Limestone/Dolomite/Marl'],
                  ['Litho_Nubian sandstone'],
                  ['FTYPE_מעביר מים'],
                  ['FTYPE_נחל צר'],
                  ['FTYPE_נחל רחב'],
                  ['FTYPE_ערוצון/בתרון'],
                  ['FTYPE_תעלה'],
                  ['Mean Moisture'],
                	['Mean Temp (C)'],
                  ['Max Temp (C)'],
                	['Min Temp (C)'],
                  ['Mean Wind Speed (M/Sec)'],
                 	['Precipitation (mm)']
                  FROM df1 JOIN df2
                     on df1.Date >= df2['Start Date'] and df1.Date <= df2.Date
                  """).df()
display(df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
del df_, df1, df2
df.columns

In [ ]:
df.drop(columns=['Date_1', 'Month', 'Year'], inplace=True)

In [ ]:
df.rename({'Longitude_x':'Longitude', 'Latitude_x':'Latitude'}, inplace=True)

## Data Aggregation

In [ ]:
aggregation_functions = {
    'Station': 'first',
    'Longitude':'first',
    'Latitude':'first',
    'Mean Moisture': 'mean',
    'Mean Temp (C)': 'mean',
    'Max Temp (C)': 'mean',
    'Min Temp (C)': 'mean',
    'Mean Wind Speed (M/Sec)': 'mean',
    'Precipitation (mm)': 'sum',
    'NDVI':'first'
    }
# Group by the Date column and apply the aggregation functions\n",

grouped_df = df.groupby(['Start Date', 'Date', 'OBJECTID', 'Slope', 'Elevation', 'STRM_ORD_1']).agg(aggregation_functions).reset_index()
display(grouped_df)

## Finding Optimal time between Rain Events and Propagation

In [ ]:
for lag in range(1, 10):
    grouped_df[f'NDVI_Lag_{lag}'] = grouped_df['NDVI'].shift(lag)

display(grouped_df)

In [ ]:
data_for_model_NDVI = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI']].iloc[:, :]
data_for_model_NDVI_Lag_1 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_1']].iloc[1:, :]
data_for_model_NDVI_Lag_2 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_2']].iloc[2:, :]
data_for_model_NDVI_Lag_3 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_3']].iloc[3:, :]
data_for_model_NDVI_Lag_4 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_4']].iloc[4:, :]
data_for_model_NDVI_Lag_5 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_5']].iloc[5:, :]
data_for_model_NDVI_Lag_6 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_6']].iloc[6:, :]
data_for_model_NDVI_Lag_7 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_7']].iloc[7:, :]
data_for_model_NDVI_Lag_8 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_8']].iloc[8:, :]
data_for_model_NDVI_Lag_9 = grouped_df[['Mean Moisture', 'Mean Temp (C)', 'Max Temp (C)', 'Min Temp (C)', 'Mean Wind Speed (M/Sec)', 'Precipitation (mm)', 'NDVI_Lag_9']].iloc[9:, :]
results = pd.DataFrame()
results['n'] = 0
results['r2'] = 0
results['mae'] = 0
results['mse'] = 0

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Split the data into features and target variable
X = data_for_model_NDVI.drop("NDVI", axis=1)
y = data_for_model_NDVI["NDVI"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[0, 'n'] = 0
results.loc[0, 'r2'] = r2
results.loc[0, 'mae'] = mae
results.loc[0, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
# Split the data into features and target variable
X = data_for_model_NDVI_Lag_1.drop("NDVI_Lag_1", axis=1)
y = data_for_model_NDVI_Lag_1["NDVI_Lag_1"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[1, 'n'] = 1
results.loc[1, 'r2'] = r2
results.loc[1, 'mae'] = mae
results.loc[1, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
# Split the data into features and target variable
X = data_for_model_NDVI_Lag_2.drop("NDVI_Lag_2", axis=1)
y = data_for_model_NDVI_Lag_2["NDVI_Lag_2"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[2, 'n'] = 2
results.loc[2, 'r2'] = r2
results.loc[2, 'mae'] = mae
results.loc[2, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
# Split the data into features and target variable
X = data_for_model_NDVI_Lag_3.drop("NDVI_Lag_3", axis=1)
y = data_for_model_NDVI_Lag_3["NDVI_Lag_3"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[3, 'n'] = 3
results.loc[3, 'r2'] = r2
results.loc[3, 'mae'] = mae
results.loc[3, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
# Split the data into features and target variable
X = data_for_model_NDVI_Lag_4.drop("NDVI_Lag_4", axis=1)
y = data_for_model_NDVI_Lag_4["NDVI_Lag_4"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[4, 'n'] = 4
results.loc[4, 'r2'] = r2
results.loc[4, 'mae'] = mae
results.loc[4, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
# Split the data into features and target variable
X = data_for_model_NDVI_Lag_5.drop("NDVI_Lag_5", axis=1)
y = data_for_model_NDVI_Lag_5["NDVI_Lag_5"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[5, 'n'] = 5
results.loc[5, 'r2'] = r2
results.loc[5, 'mae'] = mae
results.loc[5, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
# Split the data into features and target variable
X = data_for_model_NDVI_Lag_6.drop("NDVI_Lag_6", axis=1)
y = data_for_model_NDVI_Lag_6["NDVI_Lag_6"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[6, 'n'] = 6
results.loc[6, 'r2'] = r2
results.loc[6, 'mae'] = mae
results.loc[6, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

In [ ]:
display(results)

## Running again best model (N=0)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Split the data into features and target variable
X = data_for_model_NDVI.drop("NDVI", axis=1)
y = data_for_model_NDVI["NDVI"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize an XGBoost regression model
model = XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

results.loc[0, 'n'] = 0
results.loc[0, 'r2'] = r2
results.loc[0, 'mae'] = mae
results.loc[0, 'mse'] = mse

print("R-squared:", r2)
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)

## SHAP

In [ ]:
!pip install shap

In [ ]:
import shap

# SHAP analysis
explainer = shap.Explainer(model)
shap_values = explainer.shap_values(X_test)

# Plot SHAP summary plot
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
import matplotlib.pyplot as plt
# Plot SHAP scatter plots for each feature
for feature in X.columns:
    shap.dependence_plot(feature, shap_values, X_test, display_features=X_test)
    plt.show()

## Plotly Graphs

In [ ]:
df = pd.read_csv('/content/final_ds.csv')

In [ ]:
df

,Start Date,End Date,OBJECTID,Longitude,Latitude,Slope,Elevation,STRM_ORD_1,Litho_Alluvium,Litho_Chalk,...,FTYPE_נחל צר,FTYPE_נחל רחב,FTYPE_ערוצון/בתרון,FTYPE_תעלה,NDVI,Max Temp (C),Min Temp (C),Mean Moisture,Mean Wind Speed (M/Sec),Precipitation (mm)
0,2002-10-30,2002-11-13,1,34.719044,30.763107,7.235941,617.238037,0.0,False,False,...,True,False,False,False,995.0,23.733333,12.833333,69.663153,2.471250,10.0
1,2002-11-14,2002-11-28,1,34.719044,30.763107,7.235941,617.238037,0.0,False,False,...,True,False,False,False,1024.0,22.586667,9.840000,64.518388,3.019537,0.8
2,2002-11-29,2002-12-13,1,34.719044,30.763107,7.235941,617.238037,0.0,False,False,...,True,False,False,False,1188.0,18.920000,8.213333,66.037899,2.959861,10.8
3,2002-12-14,2002-12-28,1,34.719044,30.763107,7.235941,617.238037,0.0,False,False,...,True,False,False,False,1249.0,15.100000,6.740000,66.271670,3.626529,17.0
4,2002-12-29,2003-01-12,1,34.719044,30.763107,7.235941,617.238037,0.0,False,False,...,True,False,False,False,1195.0,19.573333,6.180000,68.844327,2.643011,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645640,2023-10-25,2023-11-08,1431,34.901230,30.941668,2.766679,544.985474,0.0,False,True,...,True,False,False,False,1052.0,29.960000,16.346667,57.797053,2.135463,0.0
645641,2023-11-09,2023-11-23,1431,34.901230,30.941668,2.766679,544.985474,0.0,False,True,...,True,False,False,False,1062.0,24.120000,12.733333,69.381481,2.639077,6.7
645642,2023-11-24,2023-12-08,1431,34.901230,30.941668,2.766679,544.985474,0.0,False,True,...,True,False,False,False,1157.0,21.826667,9.540000,65.406019,2.190694,10.3
645643,2023-12-09,2023-12-23,1431,34.901230,30.941668,2.766679,544.985474,0.0,False,True,...,True,False,False,False,1187.0,19.313333,8.333333,67.114481,2.888106,5.0


In [ ]:
df[['Start Date', 'NDVI', 	'Max Temp (C)' ,	'Min Temp (C)' ,	'Mean Moisture' ,	'Mean Wind Speed (M/Sec)' 	,'Precipitation (mm)']]

,Start Date,NDVI,Max Temp (C),Min Temp (C),Mean Moisture,Mean Wind Speed (M/Sec),Precipitation (mm)
0,2002-10-30,995.0,23.733333,12.833333,69.663153,2.471250,10.0
1,2002-11-14,1024.0,22.586667,9.840000,64.518388,3.019537,0.8
2,2002-11-29,1188.0,18.920000,8.213333,66.037899,2.959861,10.8
3,2002-12-14,1249.0,15.100000,6.740000,66.271670,3.626529,17.0
4,2002-12-29,1195.0,19.573333,6.180000,68.844327,2.643011,0.0
...,...,...,...,...,...,...,...
645640,2023-10-25,1052.0,29.960000,16.346667,57.797053,2.135463,0.0
645641,2023-11-09,1062.0,24.120000,12.733333,69.381481,2.639077,6.7
645642,2023-11-24,1157.0,21.826667,9.540000,65.406019,2.190694,10.3
645643,2023-12-09,1187.0,19.313333,8.333333,67.114481,2.888106,5.0


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the data
data = pd.read_csv('./final_ds.csv', usecols=['Start Date','NDVI','Max Temp (C)','Min Temp (C)','Mean Moisture','Mean Wind Speed (M/Sec)','Precipitation (mm)'])

# Convert 'Start Date' to datetime format
data['Start Date'] = pd.to_datetime(data['Start Date'])

# Extract month from 'Start Date'
data['Month'] = data['Start Date'].dt.month

# Group by month and calculate the mean
monthly_means = data.groupby('Month').mean()
monthly_means.drop(columns=['Start Date'], inplace=True)
monthly_means.reset_index(inplace=True)

# Load the data (replace this with your actual data)
data = monthly_means

# Create a Plotly figure
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Define a custom brightened royal color palette
bright_royal_palette = [
    '#00cc00', '#0000ff', '#ff0000', '#e6e600', '#ff00ff', '#ff6600']

# Add traces for each numerical column
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
for i, column in zip(range(len(numerical_columns)), numerical_columns[:-1]):
    if column != 'NDVI':
        fig.add_trace(go.Scatter(x=data['Month'], y=data[column], mode='lines', name=column, line=dict(color=bright_royal_palette[i])), secondary_y=False)
    else:
        fig.add_trace(go.Scatter(x=data['Month'], y=data[column], mode='lines', name=column, line=dict(color=bright_royal_palette[i])), secondary_y=True)

# Update layout for the plot
fig.update_layout(
    title_text='Average Monthly Distribution of Variables (2002-2023)',
    title_x=0.5,  # Center the title
    xaxis=dict(title='Month'),
    yaxis=dict(title='Value'),  # Removed y-axis tick color customization
    yaxis2=dict(title='NDVI', overlaying='y', side='right', tickfont=dict(color='forestgreen')),  # Customize secondary y-axis tick color
    legend=dict(x=0, y=-0.2, orientation='h'),  # Place legend at the bottom
    plot_bgcolor='white',  # Set plot background color
    paper_bgcolor='rgba(0,0,0,0)',  # Set paper background color (transparent)
    colorway=bright_royal_palette  # Apply the custom brightened royal color palette
)

# Show the interactive plot
fig.show()

In [ ]:
fig.write_html("file.html")